In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cab

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import warnings
import json 
import pickle
warnings.filterwarnings('ignore')

import torch
import pickle
import random

import sentence_transformers 
from sklearn.preprocessing import KBinsDiscretizer
from sentence_transformers import SentenceTransformer

from gensim.models import Word2Vec

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split, DataLoader
from torch.utils.data import Dataset

In [15]:
df_train_encoded = pd.read_csv('data/df_train_encoded.csv')
df_test_encoded = pd.read_csv('data/df_test_encoded_phase2.csv')
products_encoded = pd.read_csv('./data/products_encoded.csv')

In [16]:
df_train_encoded.shape, df_test_encoded.shape, products_encoded.shape

((3606249, 4), (316972, 4), (1410675, 14))

In [35]:
product2id = json.load(open('data/product2id.json', 'r'))
id2product = json.load(open('data/id2product.json', 'r'))
id2product = {int(k): v for k, v in id2product.items()}

In [17]:
df_train_encoded.head()

,prev_items,next_item,locale,last_item
0,"[265193, 83226]",387776,0,83226
1,"[38788, 85634, 4132, 71046]",335301,0,71046
2,"[191882, 516876, 191882, 516876, 191882, 19188...",90141,0,516876
3,"[136959, 261145, 31496, 136959, 261145, 31496,...",214540,0,469511
4,"[291068, 410614, 4219]",338089,0,4219


In [18]:
df_test_encoded.head()

,prev_items,next_item,locale,last_item
0,"[30854, 296715, 240994]",1410675,0,240994
1,"[79751, 426496, 426496]",1410675,0,426496
2,"[243200, 283983, 367639, 257230]",1410675,0,257230
3,"[170646, 280251, 165573]",1410675,0,165573
4,"[303141, 303141, 303141]",1410675,0,303141


In [19]:
df_train_encoded['prev_items'] = df_train_encoded['prev_items'].apply(eval)
df_test_encoded['prev_items'] = df_test_encoded['prev_items'].apply(eval)

In [20]:
df_train_encoded['prev_items'] = df_train_encoded.apply(lambda row: row['prev_items'] + [row['next_item']], axis=1)

In [21]:
df_train_encoded.head()

,prev_items,next_item,locale,last_item
0,"[265193, 83226, 387776]",387776,0,83226
1,"[38788, 85634, 4132, 71046, 335301]",335301,0,71046
2,"[191882, 516876, 191882, 516876, 191882, 19188...",90141,0,516876
3,"[136959, 261145, 31496, 136959, 261145, 31496,...",214540,0,469511
4,"[291068, 410614, 4219, 338089]",338089,0,4219


In [22]:
df_train_encoded.drop('next_item', axis=1, inplace=True)
df_train_encoded.drop('last_item', axis=1, inplace=True)
df_test_encoded.drop('next_item', axis=1, inplace=True)
df_test_encoded.drop('last_item', axis=1, inplace=True)

In [23]:
df_train_encoded.head()

,prev_items,locale
0,"[265193, 83226, 387776]",0
1,"[38788, 85634, 4132, 71046, 335301]",0
2,"[191882, 516876, 191882, 516876, 191882, 19188...",0
3,"[136959, 261145, 31496, 136959, 261145, 31496,...",0
4,"[291068, 410614, 4219, 338089]",0


In [24]:
df_test_encoded.head()

,prev_items,locale
0,"[30854, 296715, 240994]",0
1,"[79751, 426496, 426496]",0
2,"[243200, 283983, 367639, 257230]",0
3,"[170646, 280251, 165573]",0
4,"[303141, 303141, 303141]",0


In [25]:
df_train_encoded_exploded = df_train_encoded.explode('prev_items')
df_test_encoded_exploded = df_test_encoded.explode('prev_items')

In [28]:
df_train_encoded_exploded.insert(0, 'session_id', df_train_encoded_exploded.index)
df_test_encoded_exploded.insert(0, 'session_id', df_test_encoded_exploded.index)

In [32]:
df_train_encoded_exploded = df_train_encoded_exploded.rename(columns={'prev_items': 'item_id'})
df_test_encoded_exploded = df_test_encoded_exploded.rename(columns={'prev_items': 'item_id'})

In [40]:
df_train_encoded_exploded['item'] = df_train_encoded_exploded['item_id'].map(id2product)
df_test_encoded_exploded['item'] = df_test_encoded_exploded['item_id'].map(id2product)

In [46]:
df_train_encoded_exploded = df_train_encoded_exploded[['session_id', 'item_id', 'item', 'locale']]
df_test_encoded_exploded = df_test_encoded_exploded[['session_id', 'item_id', 'item', 'locale']]

In [66]:
df_train_encoded_exploded['item_id'] = df_train_encoded_exploded['item_id'].astype('int64')
df_test_encoded_exploded['item_id'] = df_test_encoded_exploded['item_id'].astype('int64')

In [67]:
df_train_encoded_exploded.to_csv('./data/df_train_encoded_exploded.csv', index=False)
df_test_encoded_exploded.to_csv('./data/df_test_encoded_exploded_phase2.csv', index=False)

In [52]:
df_train_encoded_exploded.head()

,session_id,item_id,item,locale
0,0,265193,B09W9FND7K,0
0,0,83226,B09JSPLN1M,0
0,0,387776,B09M7GY217,0
1,1,38788,B076THCGSG,0
1,1,85634,B007MO8IME,0


In [53]:
all_session = df_train_encoded_exploded.session_id.unique()

In [55]:
all_session

array([      0,       1,       2, ..., 3606246, 3606247, 3606248])

In [56]:
trn_idx_list = np.load('data/5fold_trn_idx_list.npy', allow_pickle=True)
val_idx_list = np.load('data/5fold_val_idx_list.npy', allow_pickle=True)

In [57]:
trn_idx_list[0]

array([      0,       1,       2, ..., 3606245, 3606246, 3606247])

In [58]:
val_idx_list[0]

array([      5,       9,      19, ..., 3606240, 3606243, 3606248])

In [59]:
df_train_encoded_exploded[df_train_encoded_exploded['session_id'].isin(trn_idx_list[0])].shape 

(15132504, 4)

In [64]:
df_train_encoded_exploded[df_train_encoded_exploded['session_id'].isin(val_idx_list[0])]

,session_id,item_id,item,locale
5,5,1967,B0749V8TC7,0
5,5,31331,B0749W93VC,0
5,5,99729,B0749TX4YP,0
5,5,27048,B0749TX4YS,0
9,9,55948,3649625660,0
...,...,...,...,...
3606248,3606248,1349818,B092S9D1SD,5
3606248,3606248,326529,B09XQQ1S72,5
3606248,3606248,165032,B0852MS7QC,5
3606248,3606248,1369560,B0B1V43MN1,5


In [65]:
df_train_encoded_exploded.dtypes

session_id     int64
item_id       object
item          object
locale         int64
dtype: object

In [62]:
15132504 + 3779928

18912432

In [63]:
df_train_encoded_exploded.shape

(18912432, 4)